In [1]:
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
from sklearn.preprocessing import MinMaxScaler



C:\Users\CasKei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# function to normalise data (?)
def normalise_x(inputX):
  min_max_scaler = preprocessing.MinMaxScaler()
  dfToNorm_scaled = min_max_scaler.fit_transform(inputX)
  x = pd.DataFrame(dfToNorm_scaled)
  return x


In [3]:
import re

# Preprocessing texts in dataset 
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'http\S+', '', text) # Remove URLs
        text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
        text = text.lower()
        # Handle self-censored words and emojis as needed
        pass
    
    else:
        return ""
    return text

In [59]:
# Read data and train test split
data = pd.read_csv('all_data.csv')
data['cleaned_text'] = data['comment_text'].apply(clean_text)
# Specify the columns to consider for toxicity
toxic_columns = ['toxicity', 'severe_toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']

# # function to normalise data (?) 
# def normalise_x(inputX): 
#     min_max_scaler = preprocessing.MinMaxScaler() 
#     dfToNorm_scaled = min_max_scaler.fit_transform(inputX) 
#     x = pd.DataFrame(dfToNorm_scaled) 
#     return x

# Create a MinMaxScaler object
scaler = preprocessing.MinMaxScaler()

# Normalize the specified columns
data[toxic_columns] = scaler.fit_transform(data[toxic_columns])

# Print the normalized data
print("Normalized Data")
print(data[toxic_columns].head())


# Update the 'toxicity' label based on the values in the specified columns
data['is_toxic'] = 0
data['is_toxic'] = data.apply(lambda row: 1 if any(row[col] >= 0.5 for col in toxic_columns) else row['is_toxic'], axis=1)

# Check the updated class distribution
print(data['is_toxic'].value_counts())

# Calculate the ratio of toxic to non-toxic samples
toxic_count = data['is_toxic'].sum()
non_toxic_count = len(data) - toxic_count
ratio = non_toxic_count / toxic_count

# If the ratio is still above a certain threshold, remove some non-toxic samples
if ratio > 2:  # Adjust the threshold as per your requirement
    non_toxic_data = data[data['is_toxic'] == 0]
    toxic_data = data[data['is_toxic'] == 1]
    
    # Randomly select a subset of non-toxic samples to keep
    num_samples_to_keep = int(toxic_count)  # Adjust the factor as per your requirement
    non_toxic_data = non_toxic_data.sample(n=num_samples_to_keep, random_state=42)
    
    # Combine the toxic and selected non-toxic samples
    data = pd.concat([toxic_data, non_toxic_data])
    
    # Shuffle the data
    data = data.sample(frac=1, random_state=42).reset_index(drop=True)

Normalized Data
   toxicity  severe_toxicity   obscene  sexual_explicit  identity_attack  \
0  0.373134         0.044776  0.089552         0.014925         0.000000   
1  0.605263         0.013158  0.065789         0.013158         0.092105   
2  0.666667         0.015873  0.031746         0.000000         0.047619   
3  0.815789         0.065789  0.552632         0.592105         0.000000   
4  0.550000         0.037500  0.337500         0.275000         0.037500   

     insult    threat  
0  0.343284  0.014925  
1  0.565789  0.065789  
2  0.666667  0.000000  
3  0.684211  0.105263  
4  0.487500  0.000000  
is_toxic
0    1835081
1     164435
Name: count, dtype: int64


In [57]:
a = np.sort(data['severe_toxicity'])
print(a)

[0.         0.         0.         ... 0.6        0.64436297 1.        ]


In [60]:
# make label binary 1 and 0 first for toxicity
data['is_toxic'] = (data['toxicity'] >= 0.5).astype(int)
 
count_zero = np.sum(data['is_toxic']  == 0) 
count_one = np.sum(data['is_toxic']  == 1)
print(count_zero, count_one)

169088 159782


In [61]:

count_zero = np.sum(data['is_toxic']  == 0)
count_one = np.sum(data['is_toxic']  == 1)

print(count_zero, count_one)
X = data['cleaned_text']
y = data['is_toxic']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [62]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


In [63]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    X_validation.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    X_test.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)


C:\Users\CasKei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [64]:
# cuda
device = torch.device("cuda")
device
import torch
print(torch.cuda.is_available())

True


In [8]:
# normalization
scalar = MinMaxScaler()

In [65]:
## convert lists to tensors

train_seq = torch.tensor(scalar.fit_transform(tokens_train['input_ids'])).long().to(device)
train_mask = torch.tensor(tokens_train['attention_mask']).to(device)
train_y = torch.tensor(y_train.tolist()).to(device)

val_seq = torch.tensor(scalar.fit_transform(tokens_val['input_ids'])).long().to(device)
val_mask = torch.tensor(tokens_val['attention_mask']).to(device)
val_y = torch.tensor(y_validation.tolist()).to(device)

test_seq = torch.tensor(scalar.fit_transform(tokens_test['input_ids'])).long().to(device)
test_mask = torch.tensor(tokens_test['attention_mask']).to(device)
test_y = torch.tensor(y_test.tolist()).to(device)



In [66]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

# wrap tensors
test_data = TensorDataset(test_seq, test_mask, test_y)

# sampler for sampling the data during training
test_sampler = SequentialSampler(test_data)

# dataLoader for testidation set
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=batch_size)


In [67]:
# freeze parameters
for param in bert.parameters():
    param.requires_grad = False


In [68]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
        super(BERT_Arch, self).__init__()
        
        self.bert = bert 
        
        # dropout layer
        self.dropout = nn.Dropout(0.2)
      
        # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768,512)
      
        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,2)

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):
        
        #pass the inputs to the model  
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
      
        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)
      
        # apply softmax activation
        x = self.softmax(x)

        return x


In [69]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)


In [70]:
# optimizer from hugging face transformers
# The compute_class_weight function from the sklearn.utils.class_weight module is used to compute the class weights with multiple parameters for the training labels.
from transformers import AdamW
from sklearn.utils.class_weight import compute_class_weight

# define the optimizer
optimizer = AdamW(model.parameters(),lr = 1e-4)

#compute the class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train.tolist()), y=y_train.tolist())

print('Class Weights:',class_weights)
print('number of weights: ', len(class_weights))
print('number of classes: ', len(np.unique(y_train.tolist())))

Class Weights: [0.97138595 1.03035097]
number of weights:  2
number of classes:  2


C:\Users\CasKei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [71]:
# converting list of class weights to a tensor
weights= torch.tensor(class_weights,dtype=torch.float)

# push to GPU
weights = weights.to(device)

# define the loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 10


In [72]:
# function to train the model
def train():
    
    model.train()
    total_loss, total_accuracy = 0, 0
  
    # empty list to save model predictions
    total_preds=[]
  
    # iterate over batches
    for step,batch in enumerate(train_dataloader):
        
        # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
        
        # push the batch to gpu
        batch = [r.to(device) for r in batch]
 
        sent_id, mask, labels = batch
        
        # clear previously calculated gradients 
        model.zero_grad()        

        # get model predictions for the current batch
        preds = model(sent_id, mask)

        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)
  
      # predictions are in the form of (no. of batches, size of batch, no. of classes).
      # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds


In [73]:
# function for evaluating the model
def evaluate(dataloader):
    
    print("\nEvaluating...")
  
    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0
    
    # empty list to save the model predictions
    total_preds = []

    # iterate over batches
    for step,batch in enumerate(dataloader):
        
        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
            
            # Calculate elapsed time in minutes.
            # elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

        # deactivate autograd
        with torch.no_grad():
            
            # model predictions
            preds = model(sent_id, mask)

            # compute the validation loss between actual and predicted values
            loss = cross_entropy(preds,labels)

            total_loss = total_loss + loss.item()

            preds = preds.detach().cpu().numpy()

            total_preds.append(preds)

    # compute the validation loss of the epoch
    avg_loss = total_loss / len(dataloader) 

    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [74]:
# set initial loss to infinite
best_valid_loss = float('inf')

#defining epochs
epochs = 5

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate(val_dataloader)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')



 Epoch 1 / 5
  Batch    50  of  8,222.
  Batch   100  of  8,222.
  Batch   150  of  8,222.
  Batch   200  of  8,222.
  Batch   250  of  8,222.
  Batch   300  of  8,222.
  Batch   350  of  8,222.
  Batch   400  of  8,222.
  Batch   450  of  8,222.
  Batch   500  of  8,222.
  Batch   550  of  8,222.
  Batch   600  of  8,222.
  Batch   650  of  8,222.
  Batch   700  of  8,222.
  Batch   750  of  8,222.
  Batch   800  of  8,222.
  Batch   850  of  8,222.
  Batch   900  of  8,222.
  Batch   950  of  8,222.
  Batch 1,000  of  8,222.
  Batch 1,050  of  8,222.
  Batch 1,100  of  8,222.
  Batch 1,150  of  8,222.
  Batch 1,200  of  8,222.
  Batch 1,250  of  8,222.
  Batch 1,300  of  8,222.
  Batch 1,350  of  8,222.
  Batch 1,400  of  8,222.
  Batch 1,450  of  8,222.
  Batch 1,500  of  8,222.
  Batch 1,550  of  8,222.
  Batch 1,600  of  8,222.
  Batch 1,650  of  8,222.
  Batch 1,700  of  8,222.
  Batch 1,750  of  8,222.
  Batch 1,800  of  8,222.
  Batch 1,850  of  8,222.
  Batch 1,900  of  8,222

In [75]:
from sklearn.metrics import classification_report

# get predictions for test data 
with torch.no_grad(): 
    preds = model(test_seq.to(device), test_mask.to(device)) 
    preds = preds.detach().cpu().numpy() 
 

# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.41 GiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 14.67 GiB is allocated by PyTorch, and 9.67 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [28]:
# import torch 
 
# # save tagged_data 
# # with open('tagged_data.pickle', 'wb') as handle: 
# #     pickle.dump(tagged_data, handle, protocol=pickle.HIGHEST_PROTOCOL) 
 
# # open and load tags from pickle file 
# model = BERT_Arch(bert)
# # model = BERT_Arch(*args, **kwargs)
# model.load_state_dict(torch.load('saved_weights2.pt'))
# model.eval()



BERT_Arch(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [76]:
model.to('cuda')
loss, tt_preds = evaluate(test_dataloader)


Evaluating...
  Batch    50  of  1,028.
  Batch   100  of  1,028.
  Batch   150  of  1,028.
  Batch   200  of  1,028.
  Batch   250  of  1,028.
  Batch   300  of  1,028.
  Batch   350  of  1,028.
  Batch   400  of  1,028.
  Batch   450  of  1,028.
  Batch   500  of  1,028.
  Batch   550  of  1,028.
  Batch   600  of  1,028.
  Batch   650  of  1,028.
  Batch   700  of  1,028.
  Batch   750  of  1,028.
  Batch   800  of  1,028.
  Batch   850  of  1,028.
  Batch   900  of  1,028.
  Batch   950  of  1,028.
  Batch 1,000  of  1,028.


In [77]:
# from sklearn.metrics import classification_report

# torch.cuda.empty_cache()
# # get predictions for test data 
# with torch.no_grad(): 
#     preds = model(test_seq.to(device), test_mask.to(device)) 
#     preds = preds.detach().cpu().numpy() 
 
 
# # model's performance 
# preds = np.argmax(preds, axis = 1) 
# print(classification_report(test_y, preds))


# tt_preds = tt_preds.to('cpu')

torch_tensor = torch.from_numpy(tt_preds)
cpu_tensor = torch_tensor.cpu()

# model's performance 
preds = np.argmax(cpu_tensor, axis = 1) 
print(classification_report(test_y.cpu(), preds))

              precision    recall  f1-score   support

           0       0.51      1.00      0.68     16809
           1       0.00      0.00      0.00     16078

    accuracy                           0.51     32887
   macro avg       0.26      0.50      0.34     32887
weighted avg       0.26      0.51      0.35     32887



C:\Users\CasKei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CasKei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CasKei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1

In [39]:
print(tt_preds)

[[-2.073747   -0.13434753]
 [-0.7725483  -0.61958945]
 [-0.29789892 -1.3562554 ]
 ...
 [-0.12222891 -2.1623516 ]
 [-0.03088112 -3.4930108 ]
 [-0.35752735 -1.2019867 ]]


In [48]:
print(test_y[22])
print(tt_preds[22])

tensor(1, device='cuda:0')
[-0.54296535 -0.86993873]
